In [1]:
import nbimporter
import functions
reload(functions)
import pandas as pd
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
import numpy as np
#viz
from datetime import date
from random import randint
import math

from bokeh.models import ColumnDataSource
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_file, show
from bokeh.models.tickers import FixedTicker

from sklearn import ensemble
from sklearn.metrics.pairwise import linear_kernel
from dateutil import parser

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
from matplotlib import pyplot

#from bokeh.io import vform

Importing Jupyter notebook from functions.ipynb
Importing Jupyter notebook from functions.ipynb


/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
##variables
look_back = 2

In [3]:
#functions.test()

In [4]:
log_file = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file1 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_7.csv'
log_file2 = '/Users/momori/Documents/projects/log_monitoring/muleprod_app_aug_1.csv'
log_file3 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'
log_file4 = '/Users/momori/Documents/projects/log_monitoring/muleprep_app_ERROR.csv'

df_test = pd.read_csv(log_file)
df_train1 = pd.read_csv(log_file)#1)
df_train2 = pd.read_csv(log_file)#2)

/Users/momori/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,4,5,6,7,8,10,11,12,13,14,15,18,19,20,21,22,23,25,26,28,29,30,31,33,34,36,38,40,41,42,45,46,48,49,50,52,53,54,55,56,57,59,60,63,64,65,68,69,70,72,73,74,75,76,77,79,80,81,82,83,87,89,90,91,100,101,102,105,107,108,109,110,111,112,114,115,116,117,119,122,123,124,126,127,128,129,130,131,133,135,136,137,139,142,143,144,145,146,148,149,150,154,155,156,158,159,160,161,169,171,172,173,174,175,176,177,178,179,181,183,184,185,186,194,196,198,200,201,202,204,209,214,217,218,219,220,224,225,227,229,230,231,232,233,234,235,238,242,243) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train_df = [df_train1, df_train2]

## Functions

In [6]:
##km: clustering model
##vecetorizer: document vectorizer
##document: document to cluster
def predict_document_cluster(km, vectorizer, document):
    doc = vectorizer.transform([document])
    return km.predict(doc)

def format_df(df):
    r_df = df.dropna(axis=1)
    r_df.loc[:,'_time'] = r_df['_time'].apply(lambda x: format_time(x))
    return r_df

def format_time(string):
    ##may need smarter logic. seems slow
    return parser.parse(string)

##step_time in minutes
##
## returns: dict. key is timestamp, value is dataframe
def split_by_time(df, step_time=30):
    r_dict = {}
    t=str(step_time)+'Min'
    
    #set _time to index
    tmp = df.set_index(1)
    tmp = tmp.groupby(pd.TimeGrouper(freq=t))
    for key, item in tmp:
        try:
            r_dict[str(key)] = tmp.get_group(key)
        except:
            continue
    return r_dict
        
def show_top_words(km, vect, show=False, save=True, out_filename='output'):
    dcluster_words = {}
    if show:
        print("Top terms per cluster:")

    order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vect.get_feature_names()
    for i in range(k):
        if show:
            print("Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])
        if save:
            words = []
            for ind in order_centroids[i, :10]:
                words.append(terms[ind])
            dcluster_words[i] = words
    if save:
        np.save(out_filename, dcluster_words)


In [7]:
#show_top_words(km, vect, save=True)
!ls

Untitled.ipynb               lowerbound.npy
Untitled1.ipynb              module_test.ipynb
bollinger.html               new_df_formatted
check_logs                   new_logs
cluster_num.npy              output.npy
cluster_words                splunk log analyzer v2.ipynb
counts.npy                   splunk log analyzer.ipynb
data_table.html              splunk_log_analyzer_v3.ipynb
functions.ipynb              splunk_viz.ipynb
intuit.html                  times
line_bar.html                times.npy
log_analysis_tutorial.ipynb  upperbound.npy
log_cleaning.ipynb


In [8]:
# d = np.load('output.npy').item()
# for k,v1 in d.iteritems():
#     print k,v1

In [9]:
logs_train1 = df_train1['_raw'].tolist()
logs_train2 = df_train2['_raw'].tolist()
logs_test = df_test['_raw'].tolist()

In [10]:
##Fit a vectorizer
vect = TfidfVectorizer(max_df = 0.5, min_df = 2, stop_words='english')

#v is base vectorizer, can be used to transform other logs
v = vect.fit(logs_train1)

#x is the tfidf sparse matrix created for this set of logs
x1 = v.transform(logs_train1)
x2 = v.transform(logs_train2)

In [11]:
##create kmeans model and fit
k=20
km = MiniBatchKMeans(n_clusters=k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000)
km.fit(x1)

MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=1000, max_iter=100, max_no_improvement=10, n_clusters=20,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=0)

In [12]:
##Let's use cluster 6 as it shows '500' as one of the top words
show_top_words(km, vect, show=True)

Top terms per cluster:
Cluster 0:
 java
 component
 groovy
 scriptcomponent
 common
 codehaus
 accounts
 903970365
 v1
 eval
Cluster 1:
 entitlement
 commerce
 01
 entitlementofferingrestransformer
 entitledoffering
 connectorwithoutmulesession
 cas
 transformer
 services
 received
Cluster 2:
 accountconversionapi
 123146541871034
 internal
 websenhanceddataexception
 414
 exist
 does
 requested
 resource
 cdata
Cluster 3:
 grizzly
 glassfish
 safefutureimpl
 singleendpointpool
 java
 abstractthreadpool
 impl
 thread
 dowork
 connectionpool
Cluster 4:
 catalog
 offers
 invokerestorchestration
 txid
 offer
 connector
 flow
 default
 app
 karthik_iamtestpass_710283333
Cluster 5:
 followredirects
 redirect
 httpclientmessagedispatcher
 500
 httpsconnector
 internal
 router
 server
 tbt
 transport
Cluster 6:
 jar
 core
 execute
 execution
 java
 exceptiontomessagingexceptionexecutioninterceptor
 messageprocessornotificationexecutioninterceptor
 blockingprocessorexecutor
 process
 messagepr

In [13]:
#for i in range(20):
#    print i, len(df_predict1[df_predict1[2]==i]), len(df_predict2[df_predict2[2]==i])

In [14]:
##get predictions for a subset of the logs and create df with _raw, _time, and label
formatted_df = format_df(df_train1)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = v.transform(logs)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
df_predict1 = pd.DataFrame(lst)
df_predict1.head()

##do same for second train set
formatted_df = format_df(df_train2)

logs = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x2 = v.transform(logs)

labels = km.predict(test_x2)
lst = zip(logs, times, labels.tolist())
df_predict2 = pd.DataFrame(lst)
df_predict2.head()


/Users/momori/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,0,1,2
0,"INFO 2017-08-19 14:18:11,732 [[ebcs-restricti...",2017-08-19 14:18:11.732000-07:00,8
1,"2017-08-19 14:18:09,124 | level=ERROR| thread=...",2017-08-19 14:18:09.124000-07:00,3
2,"2017-08-19 14:17:48,122 | level=ERROR| thread=...",2017-08-19 14:17:48.122000-07:00,3
3,"INFO 2017-08-19 14:17:47,376 [[webs-pub-conve...",2017-08-19 14:17:47.376000-07:00,18
4,"INFO 2017-08-19 14:17:47,374 [[webs-pub-conve...",2017-08-19 14:17:47.374000-07:00,18


In [15]:
df_predict1.shape

(118716, 3)

In [16]:
##for each cluster number, create a dt
#df_clusters1, df_clusters2 = None


In [17]:
cluster_num = 8
f_df1 = df_predict1[df_predict1[2]==cluster_num]
f_df2 = df_predict2[df_predict2[2]==cluster_num]
#f_df1 = df_predict1
#f_df2 = df_predict2
f_df1.head()

,0,1,2
0,"INFO 2017-08-19 14:18:11,732 [[ebcs-restricti...",2017-08-19 14:18:11.732000-07:00,8
667,"INFO 2017-08-19 14:07:15,740 [[ebcs-restricti...",2017-08-19 14:07:15.740000-07:00,8
668,"INFO 2017-08-19 14:07:11,816 [[ebcs-restricti...",2017-08-19 14:07:11.816000-07:00,8
724,"INFO 2017-08-19 14:06:05,157 [[ebcs-restricti...",2017-08-19 14:06:05.157000-07:00,8
2530,"INFO 2017-08-19 13:35:53,037 [[ebcs-restricti...",2017-08-19 13:35:53.037000-07:00,8


In [18]:
check_logs = f_df1[0].tolist()

In [19]:
check_logs

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [20]:
##Create dictionary of dataframes, split by time
d1 = OrderedDict(sorted(split_by_time(f_df1).items()))
d2 = OrderedDict(sorted(split_by_time(f_df2).items()))
#d1 = OrderedDict(sorted(split_by_time(df_predict1).items()))
#d2 = OrderedDict(sorted(split_by_time(df_predict2).items()))


In [21]:
#create counts for training
counts1 = []
counts2 = []
for k, val in d1.iteritems():
    counts1.append(val.shape[0])
for k, val in d2.iteritems():
    counts2.append(val.shape[0])

In [22]:
len(counts1), len(counts2)

(48, 48)

In [23]:
counts1[:10], counts2[:10]

([11, 231, 342, 152, 64, 264, 56, 33, 56, 72],
 [11, 231, 342, 152, 64, 264, 56, 33, 56, 72])

In [24]:
v = vect.fit(logs_train1)
v.transform(logs_test)

<118716x56347 sparse matrix of type '<type 'numpy.float64'>'
	with 6591783 stored elements in Compressed Sparse Row format>

In [25]:
##Let's fit the 'future' data and pull out cluster_num
formatted_df = format_df(df_test)
logs_test = formatted_df['_raw'].tolist()
times = formatted_df['_time'].tolist()
test_x1 = v.transform(logs_test)

labels = km.predict(test_x1)
lst = zip(logs, times, labels.tolist())
test_data = pd.DataFrame(lst)
test_data.head()

,0,1,2
0,"INFO 2017-08-19 14:18:11,732 [[ebcs-restricti...",2017-08-19 14:18:11.732000-07:00,8
1,"2017-08-19 14:18:09,124 | level=ERROR| thread=...",2017-08-19 14:18:09.124000-07:00,3
2,"2017-08-19 14:17:48,122 | level=ERROR| thread=...",2017-08-19 14:17:48.122000-07:00,3
3,"INFO 2017-08-19 14:17:47,376 [[webs-pub-conve...",2017-08-19 14:17:47.376000-07:00,18
4,"INFO 2017-08-19 14:17:47,374 [[webs-pub-conve...",2017-08-19 14:17:47.374000-07:00,18


In [26]:
test_data = test_data[test_data[2]==cluster_num]

#test_data = test_data
test_d = OrderedDict(sorted(split_by_time(test_data).items()))
test_counts = []
for k, val in test_d.iteritems():
    test_counts.append(val.shape[0])


In [27]:
test_data.head()

,0,1,2
0,"INFO 2017-08-19 14:18:11,732 [[ebcs-restricti...",2017-08-19 14:18:11.732000-07:00,8
667,"INFO 2017-08-19 14:07:15,740 [[ebcs-restricti...",2017-08-19 14:07:15.740000-07:00,8
668,"INFO 2017-08-19 14:07:11,816 [[ebcs-restricti...",2017-08-19 14:07:11.816000-07:00,8
724,"INFO 2017-08-19 14:06:05,157 [[ebcs-restricti...",2017-08-19 14:06:05.157000-07:00,8
2530,"INFO 2017-08-19 13:35:53,037 [[ebcs-restricti...",2017-08-19 13:35:53.037000-07:00,8


In [28]:
len(test_counts)

48

In [29]:
test_counts[:10]

[11, 231, 342, 152, 64, 264, 56, 33, 56, 72]

In [30]:
#zip the data
counts1 = [float(i) for i in counts1]
counts2 = [float(i) for i in counts2]

train = zip(counts1, counts2)
train = [i for i in train if math.fabs(i[1]-i[0] < 100)]
print len(train)

48


In [31]:
train

[(11.0, 11.0),
 (231.0, 231.0),
 (342.0, 342.0),
 (152.0, 152.0),
 (64.0, 64.0),
 (264.0, 264.0),
 (56.0, 56.0),
 (33.0, 33.0),
 (56.0, 56.0),
 (72.0, 72.0),
 (273.0, 273.0),
 (36.0, 36.0),
 (147.0, 147.0),
 (247.0, 247.0),
 (47.0, 47.0),
 (54.0, 54.0),
 (10.0, 10.0),
 (11.0, 11.0),
 (166.0, 166.0),
 (310.0, 310.0),
 (177.0, 177.0),
 (100.0, 100.0),
 (49.0, 49.0),
 (330.0, 330.0),
 (231.0, 231.0),
 (17.0, 17.0),
 (179.0, 179.0),
 (356.0, 356.0),
 (222.0, 222.0),
 (291.0, 291.0),
 (27.0, 27.0),
 (2.0, 2.0),
 (8.0, 8.0),
 (2.0, 2.0),
 (1.0, 1.0),
 (3.0, 3.0),
 (1.0, 1.0),
 (1.0, 1.0),
 (6.0, 6.0),
 (28.0, 28.0),
 (1.0, 1.0),
 (11.0, 11.0),
 (3.0, 3.0),
 (7.0, 7.0),
 (22.0, 22.0),
 (15.0, 15.0),
 (1.0, 1.0),
 (4.0, 4.0)]

## ARIMA prediction

In [32]:
##make predictions by fitting ARIMA model into the lists
prediction_upperbound = []
prediction_lowerbound = []
prediction_value = []

for t in train:
    rgr = ARIMA(t, order=(0,0,0))
    fit = rgr.fit(disp=0)
    forecast =  fit.forecast()
    prediction_upperbound.append(forecast[2][0][1])
    prediction_lowerbound.append(forecast[2][0][0])
    prediction_value.append(forecast[0])

#Turn into numpy array for computation
prediction_upperbound = np.asarray(prediction_upperbound)
prediction_lowerbound = np.asarray(prediction_lowerbound)
prediction_value = np.asarray(prediction_value)

/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/momori/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization f

## Vis

In [33]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = prediction_upperbound
lowerband = prediction_lowerbound
x_data = np.arange(1, 101)

length = min(len(prediction_upperbound), len(prediction_lowerbound))
x_data = np.arange(1, length)
# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="")

# add a line renderer
p.line([i for i in range(1,length)], counts2, line_width=2)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

show(p)

/Users/momori/anaconda2/lib/python2.7/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 47), ('y', 48)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/momori/anaconda2/lib/python2.7/site-packages/bokeh/models/sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 94), ('y', 96)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


In [ ]:
f = '/Users/momori/PycharmProjects/AIMonitoring/data/qq.csv'
dd = pd.read_csv(f)

In [ ]:
dd

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = prediction_upperbound
lowerband = prediction_lowerbound
x_data = np.arange(1, 101)

length = min(len(prediction_upperbound), len(prediction_lowerbound))
x_data = np.arange(1, length)
# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="")

# add a line renderer
p.line([i for i in range(1,length)], counts2, line_width=2)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)




p.xaxis.ticker = FixedTicker(ticks=times)




show(p)

/Users/momori/anaconda2/lib/python2.7/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 13), ('y', 14)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/momori/anaconda2/lib/python2.7/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('x', 26), ('y', 28)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


# Test

In [ ]:
##get predictions for a subset of the logs and create df with _raw, _time, and label
test_df = format_df(df[0:10000])

logs = test_df['_raw'].tolist()
times = test_df['_time'].tolist()
test_x = vect.transform(logs)

labels = km.predict(test_x)
lst = zip(logs,times,labels.tolist())
df_predict = pd.DataFrame(lst)
df_predict

In [ ]:
##Create dictionary of dataframes, split by time
d = split_by_time(df_predict)
for k,v in d.iteritems():
    print k, ':::', type(v)
    

In [ ]:
d['2017-08-19 13:00:00-07:00']

In [ ]:
##Cluster the interval logs by using km.  
#Then get the count for predicting outcomes.

#test out on d
counts = []
for k, v in d.iteritems():
    print k
    counts.append(v.shape[0])



In [ ]:
counts

In [ ]:
##train gradient boosted trees for regression
params = {'n_estimators': 10, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)
clf

## Use an ARIMA Model to predict

In [ ]:
floats = [float(i) for i in counts]
floats

In [ ]:
rgr = ARIMA(floats[:len(floats)-1], order=(0,0,0))

In [ ]:
fit = rgr.fit(disp=0)

In [ ]:
fit.forecast()

## test area

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

# Define Bollinger Bands.
upperband = np.random.random_integers(100, 150, size=100)
lowerband = upperband - 100
x_data = np.arange(1, 101)

# Bollinger shading glyph:
band_x = np.append(x_data, x_data[::-1])
band_y = np.append(lowerband, upperband[::-1])

output_file('bollinger.html', title='Bollinger bands (file)')


p = figure(x_axis_type='datetime', title="Bollinger Bands")

# add a line renderer
p.line([0, 20, 30, 40, 50], [80, 130, 135, 90, 90], line_width=4)
p.grid.grid_line_alpha = 0.4
p.x_range.range_padding = 0
p.plot_height = 600
p.plot_width = 800

p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

show(p)

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models.ranges import Range1d
# import numpy


# output_file("line_bar.html")

# p = figure(plot_width=400, plot_height=400)

# # add a line renderer
# p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)

# # setting bar values
# h = numpy.array([2, 8, 5, 10, 7])

# # Correcting the bottom position of the bars to be on the 0 line.
# adj_h = h/2

# # add bar renderer
# p.rect(x=[1, 2, 3, 4, 5], y=adj_h, width=0.4, height=h, color="#CAB2D6")

# # Setting the y  axis range   
# p.y_range = Range1d(0, 12)

# #p.title = "Line and Bar"

# show(p)

In [ ]:
# from bokeh.plotting import figure, output_file, show
# from bokeh.models.ranges import Range1d
# import numpy


# output_file("line_bar.html")
# p = figure(plot_width=400, plot_height=400)

# p.line([1, 2, 3, 4, 5], [6, 7, 6, 4, 5], line_width=2)


# show(p)